# Machine Learning

Machine Learning refers to the area of Artificial Intelligence where computers rely on algorithms and statistical models to make inferences without relying on explicit instructions. This is done by studying patterns in data, designing features that represent the patterns as best as possible and finally, using decision-based rules to determine the outcome. Machine Learning can be generally grouped under 2 broad groups - supervised and unsupervised leaning. Unsupervised learning studies data by grouping them into clusters based on some similarity metrics and generally deals with data with unknown labels i.e. different customer profiles in a gaming industry etc. Supervised learning, on the other hand, learns a function that maps an input to an output based on example input-output pairs. It infers a function from labeled training data consisting of a set of training examples.

In this notebook, we will be looking at supervised learning. Supervised learning outcomes can be broadly categorized into whether we are interested in outputs as continuous values (regression) or as labels/categories (classification). An example of regression would be to predict the price of a stock (continuous variable) 2 days from now based on historical data till today. An example of classification of classification would be to predict whether it will rain tomorrow or not (labels: rain or no rain). 

## Regression

We will explore regression in this section using the HDB resale data we created earlier. Specifically, consider the following scenario:

### Scenario

A database administrator mistakenly deleted the resale flat price data for some transactions that took place in 2018. HDB is interested in the use of statistical and/or machine learning techniques to best represent the missing data.

Assumptions:
- All data on resale price are missing in 2018.
- Train your model based on data up to and including the month value of “2017-12”, then use your model to predict flat type from “2018-01” onwards

Now, since resale price is a continuous variable, we will use regression to predict what the prices are. The data does contain the ground truth which we will use to assess how close our predicted prices are compared to the actual prices. In order to predict the resale price (the dependent variable), we first need to find a set of suitable predictor variables (also known as independent variables) from the dataset. 

### So... what is regression?

Regression analysis is a set of statistical processes for estimating the relationships among variables. It includes many techniques for modeling and analyzing several variables, when the focus is on the relationship between a dependent variable and one or more independent variables (or 'predictors'). Regression analysis helps one understand how the typical value of the dependent variable changes when any one of the independent variables is varied, while the other independent variables are held fixed.

Regression models involve the following parameters and variables:

- The unknown parameters, denoted as $\beta$, which may represent a scalar or a vector.
- The independent variables, $X$.
- The dependent variable, $Y$.

A regression model relates $Y$ to a function of $X$ and $\beta$.

$$\displaystyle Y\approx f(X,\beta )$$

### Linear Regression

Linear regression assumes that there is approximately a linear relationship between the predictor variables and the dependent variable. Suppose that there are $p$ distinct predictors. Then the linear regression model takes the form
$$Y = \beta_0 + \beta_{1}X_1 + \beta_{2}X_2 + · · · + \beta_{p}X_p,$$
The coefficients ($\beta_0,\beta_1,...,\beta_p$) of the regression equation are unknown. We can use the data to estimate them. Let $(x_1, y_1), (x_2, y_2), . . . , (x_n, y_n)$ represent $n$ observation pairs, each of which consists of a measurement of $X$ and a measurement of $Y$. Our goal is to obtain coefficient estimates $\hat{\beta}_0$ to $\hat{\beta}_p$ such that the linear model fits the available data well so that $y_i ≈ \hat{\beta}_0 + \hat{\beta}_{1}x_{i1} + \hat{\beta}_{2}x_{i2} + · · · + \hat{\beta}_{p}x_{ip}$ for $i = 1, . . . , n$. In other words, we want to find an intercept $\hat{\beta}_0$ and parameters $\hat{\beta}_1$ to $\hat{\beta}_p$ such that the resulting hyperplane is as close as possible to the $n$ data points. There are a number of ways of measuring closeness. However, by far the most common approach involves minimizing the __least squares__ criterion.
![Linear Regression](images/Linear_Regression.jpg)
#### Least Squares 

Let $\hat{y}_i = \hat{\beta}_0 + \hat{\beta}_{1}x_{i1} + \hat{\beta}_{2}x_{i2} + · · · + \hat{\beta}_{p}x_{ip},$ be the prediction for Y based on the $i$-th value of X. Then 
$$e_i = y_i−\hat{y}_i$$
represents the $i$-th residual — this is the difference between the $i$-th observed response value and the $i$-th response value that is predicted by our linear model. We define the *residual sum of squares (RSS)* as
$$RSS = e^{2}_{1}+ e^{2}_{2} + · · · + e^{2}_{n}$$
RSS measure the total error between the actual responses and predicted responses. Naturally, the smaller the total error, the better is our estimation for the regression parameters. Hence, the RSS function is our optimization function that we are looking to minimize. The values of the regression paramters that provides the minimum value are the optimal values for the function. The regression coefficient estimates have somewhat complicated forms that are most easily represented using matrix algebra. For this reason, we will not discuess them here. Any statistical software package can be used to compute these coefficient estimates (which will be shown later).

#### So let's run regression on the data to answer the question...

The first step is to import the required Python libraries.

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
DATA_DIR = './Data/'

Load the data from the earlier notebook. Make sure the column data type is in the correct format, especially the ones that contain date-time information.

In [ ]:
# Read CSV file and inspect columns
df = pd.read_csv(DATA_DIR+'HDB_Resale_Data_cleaned.csv',dtype= {'Postal Code': str})
df['month'] = pd.to_datetime(df['month'], format="%Y-%m-%d")
df['lease_commence_date'] = pd.to_datetime(df['lease_commence_date'], format="%Y-%m-%d")
df.columns

All the columns except for "resale_price" (which is the dependent variable) are candidates for independent variables. As the data spans from 1990s to 2018, it may not be wise to use the data from so long back to do predict data in 2018. For this study, using the data of 5 years preceding 2018, will be sufficient for prediction. The data set is a mix of numerical and string values. All inputs to regression have to be numerical in value. The string values in the data are categorical (as in distict values and not continuous). There is an approach called One-Hot encoding for categorical data which basically creates a numerical representation for the different categories. In Pandas, this can be done by calling *dummies* function. The data frame can the be split into train and test sets.

#### Using all columns as independent variables
Excluding the "resale_price" column, let's use all other columns as features for linear regression.

In [ ]:
df_all = df
df_all['Postal Code'] = pd.to_numeric(df_all['Postal Code'])
df1 = pd.get_dummies(df_all)
df1.columns

After one-hot encoding the non-numerical categorical columns, the number of independent variable has ballooned from 12 to 8904. Now, split the data into its respective train and test sets.

In [ ]:
df_train = df1[(df1['month'].dt.year > 2012) & (df1['month'].dt.year < 2018)]
df_train['lease_commence_date'] = pd.to_numeric(df_train['lease_commence_date'])
df_train['month'] = pd.to_numeric(df_train['month'])
y_train = df_train['resale_price']
df_train = df_train.drop(['resale_price'], axis = 1)
df_train.head()

Can you create the test set? Name the test dataframe as df_test and the groud truth target as y_test.

In [ ]:
## To do
## Prepare the test set (2018 data) from the one-hot encoded dataframe (df1)
## Hint: you can use the lines from the cell above with some modifications


In [ ]:
# Solution
df_test = df1[(df1['month'].dt.year == 2018)]
df_test['lease_commence_date'] = pd.to_numeric(df_test['lease_commence_date'])
df_test['month'] = pd.to_numeric(df_test['month'])
y_test = df_test['resale_price']
df_test = df_test.drop(['resale_price'], axis = 1)
df_test.head()

Initialize the regression model and fit it on the training data.

In [ ]:
lm = LinearRegression(normalize=False,n_jobs=-1)

In [ ]:
lm.fit(df_train,y_train)

Once the model is trained, how to assess the fit of the regression model? A well-fitting regression model results in predicted values close to the observed data values. Three statistics are used in regression to evaluate model fit: R-squared, the overall F-test, and the Root Mean Square Error (RMSE). All three are based on two sums of squares: Sum of Squares Total (SST) and Sum of Squares Error (SSE). SST measures how far the data are from the mean, and SSE measures how far the data are from the model’s predicted values. Different combinations of these two values provide different information about how the regression model compares to the mean model. Here, let's just focus on R-squared and Root Mean Square Error (RMSE).

#### R-squared and Adjusted R-squared

The difference between SST and SSE is the improvement in prediction from the regression model, compared to the mean model. Dividing that difference by SST gives R-squared. It is the proportional improvement in prediction from the regression model, compared to the mean model. It indicates the goodness of fit of the model. It is interpreted as the proportion of total variance that is explained by the model.

R-squared has the useful property that its scale is intuitive: it ranges from zero to one, with zero indicating that the proposed model does not improve prediction over the mean model, and one indicating perfect prediction. Improvement in the regression model results in proportional increases in R-squared.

One pitfall of R-squared is that it can only increase as predictors are added to the regression model. This increase is artificial when predictors are not actually improving the model’s fit. To remedy this, a related statistic, Adjusted R-squared, incorporates the model’s degrees of freedom. Adjusted R-squared will decrease as predictors are added if the increase in model fit does not make up for the loss of degrees of freedom. Likewise, it will increase as predictors are added if the increase in model fit is worthwhile.

#### RMSE

The RMSE is the square root of the variance of the residuals (RSS). It indicates the absolute fit of the model to the data – how close the observed data points are to the model’s predicted values. Whereas R-squared is a relative measure of fit, RMSE is an absolute measure of fit. As the square root of a variance, RMSE can be interpreted as the standard deviation of the unexplained variance, and has the useful property of being in the same units as the response variable. Lower values of RMSE indicate better fit. RMSE is a good measure of how accurately the model predicts the response, and it is the most important criterion for fit if the main purpose of the model is prediction.

In [ ]:
# The mean squared error
print("Root mean squared error: %.2f" % np.sqrt(mean_squared_error(y_train, lm.predict(df_train))))
# Explained variance score: 1 is perfect prediction
print("R^{2} score: ", lm.score(df_train,y_train))
# Adjusted R-Squared
print("Adjusted R^{2} score: ", 1 - (1-lm.score(df_train,y_train))*(len(y_train)-1)/(len(y_train)-df_train.shape[1]-1))

In [ ]:
print("Estimated intercept coefficient: ", lm.intercept_)

In [ ]:
# Create dataframe of regression coefficients and sort values from highest to lowest
reg_coef = pd.DataFrame(zip(df_train.columns, lm.coef_), columns=['features','estimatedCoefficients']).sort_values(['estimatedCoefficients'],ascending=False) 
reg_coef.head(10)

In [ ]:
# Make predictions using the testing set
y_pred = lm.predict(df_test)

In [ ]:
# The mean squared error
print("Root mean squared error: %.2f" % np.sqrt(mean_squared_error(y_test, y_pred)))
# Explained variance score: 1 is perfect prediction
print('R^{2} score:', lm.score(df_test, y_test))
# Adjusted R-Squared
print("Adjusted R^{2} score: ", 1 - (1-lm.score(df_test, y_test))*(len(y_test)-1)/(len(y_test)-df_test.shape[1]-1))

In [ ]:
df_predict_gt = pd.DataFrame({'Predicted Price' : y_pred, 'Actual Price' : y_test})
df_predict_gt.head()

The RMSE, R-squared and Adjusted R-squared values are mostly poor for both the the trained model and on the test data. The RMSE values are above $100000 which is more than 10\% of the mean of the resale price. This indicates quite a significant error. Similarly, the low values of the R-squared and Adjusted R-squared values are indicative of the poor fit of the regression model. 

This can be explained by the fact that not all the independent variables are useful for predicting resale price. With a bit of understanding, we can select a few columns that we best think affects resale prices.

#### Feature Selection

Let's try selecting only some columns from the data to create our regression model. Let's take the following columns - 'flat_model','flat_type','floor_area_sqm','storey_range','remaining_lease','town','Postal Code' - and see if the regression model is better.

In [ ]:
df1 = df[['flat_model','flat_type','floor_area_sqm','month','storey_range','remaining_lease','town','Postal Code','resale_price']]
df1.columns

In [ ]:
df1['Postal Code'] = pd.to_numeric(df1['Postal Code'])
df1 = pd.get_dummies(df1)
df1.columns

Create the train and test dataframes from df1 above with the associated target variables. You can refer to the previous cells to figure out.

In [ ]:
## To do
## Create the train dataframe (df_train) and target variable (y_train)


In [ ]:
## To do
## Create the test dataframe (df_test) and target variable (y_test)


In [ ]:
# Solution (Train)
df_train = df1[(df1['month'].dt.year > 2012) & (df1['month'].dt.year < 2018)]
y_train = df_train['resale_price']
df_train = df_train.drop(['resale_price','month'], axis = 1)
df_train.head()

In [ ]:
# Solution (Test)
df_test = df1[(df1['month'].dt.year == 2018)]
y_test = df_test['resale_price']
df_test = df_test.drop(['resale_price','month'], axis = 1)
df_test.head()

In [ ]:
lm = LinearRegression(normalize=False,n_jobs=-1)

In [ ]:
lm.fit(df_train,y_train)

In [ ]:
# The mean squared error
print("Root mean squared error: %.2f" % np.sqrt(mean_squared_error(y_train, lm.predict(df_train))))
# Explained variance score: 1 is perfect prediction
print("R^{2} score: ", lm.score(df_train,y_train))
# Adjusted R-Squared
print("Adjusted R^{2} score: ", 1 - (1-lm.score(df_train,y_train))*(len(y_train)-1)/(len(y_train)-df_train.shape[1]-1))

In [ ]:
print("Estimated intercept coefficient: ", lm.intercept_)

In [ ]:
# Create dataframe of regression coefficients and sort values from highest to lowest
reg_coef = pd.DataFrame(zip(df_train.columns, lm.coef_), columns=['features','estimatedCoefficients']).sort_values(['estimatedCoefficients'],ascending=False) 
reg_coef.head(30)

In [ ]:
# Make predictions using the testing set
y_pred = lm.predict(df_test)

In [ ]:
# The mean squared error
print("Root mean squared error: %.2f" % np.sqrt(mean_squared_error(y_test, y_pred)))
# Explained variance score: 1 is perfect prediction
print('R^{2} score:', lm.score(df_test, y_test))
# Adjusted R-Squared
print("Adjusted R^{2} score: ", 1 - (1-lm.score(df_test, y_test))*(len(y_test)-1)/(len(y_test)-df_test.shape[1]-1))

In [ ]:
df_predict_gt = pd.DataFrame({'Predicted Price' : y_pred, 'Actual Price' : y_test})
df_predict_gt.head()

Now, selecting columns actually improved the linear regression model significantly. The RMSE value is halved by more than 50\% and the R-squared and Adjusted R-squared have improved to 0.85 for both the train and test set. Also, looking at the predicted prices and the actual price, it is obvious that the error is quite small. 

But is it possible to improve this even further? Let's try non-linear regression!

### Non-Linear Regression
The above assumes that there exists a linear correlation between the dependent variable and independent variables. However, the $R^{2}$ value is about 0.85. For a perfect linear correlation, the value will be very close to or will be at one. Now, let's assume the relation is non-linear now. 

The mathematics behind non-linear regression is a bit complex to explain in this workshop. Instead, we will just apply a non-linear regression approach on the data and observe the diffence in the results.

#### Random Forest Regression
Random forest is a type of supervised machine learning algorithm based on ensemble learning. Ensemble learning is a type of learning where you join different types of algorithms or same algorithm multiple times to form a more powerful prediction model. The random forest algorithm combines multiple algorithm of the same type i.e. multiple decision trees, resulting in a forest of trees, hence the name "Random Forest". The random forest algorithm can be used for both regression and classification tasks.
![Random Forest Structure](images/Random_Forest_Regression.jpg)
The following are the basic steps involved in performing the random forest algorithm:
- Pick N random records from the dataset.
- Build a decision tree based on these N records.
- Choose the number of trees you want in your algorithm and repeat steps 1 and 2.
- In case of a regression problem, for a new record, each tree in the forest predicts a value for Y (output). The final value can be calculated by taking the average of all the values predicted by all the trees in forest.

#### So let's run random forest regression on the data to answer the question...
The first step is to import the required Python libraries.

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

Now, we will use the same features as we did in the feature selection for linear regression so that we can compare with minimal bias. So, let's just start the training process.

The function below evaluates the performance of the model.

In [ ]:
def evaluate(model, features, labels):
    predictions = model.predict(features)
    rmse = np.sqrt(mean_squared_error(labels, predictions))
    r2 = r2_score(labels, predictions)
    adjusted_r2 = 1 - (1-r2_score(labels, predictions))*(len(labels)-1)/(len(labels)-features.shape[1]-1)
    print("Root mean squared error: %.2f" % rmse)
    # Explained variance score: 1 is perfect prediction
    print('R^{2} score: %.2f' % r2)
    print('Adjusted R^{2} score: %.2f' % adjusted_r2)
    return predictions,rmse,r2,adjusted_r2

The Random Forest regressor has many hyperparameters that require tuning to ensure good performance.

In [ ]:
rf_eg = RandomForestRegressor(random_state = 42)

from pprint import pprint
# Look at parameters used by random forest
print('Parameters currently in use:\n')
pprint(rf_eg.get_params())

Ignoring 'verbose' and 'random_state', there are 14 hyperparameters to tune for good performance. In order to find the best parameters, a grid search on a range of values for each hyperparameter should be performed with a $k$-fold cross-validation on the training set and the parameters that return the lowest error will be used to train and test the model on the test set. 

#### Cross-Validation
Cross-validation is a resampling procedure used to evaluate machine learning models on a limited data sample.

The procedure has a single parameter called $k$ that refers to the number of groups that a given data sample is to be split into. As such, the procedure is often called $k$-fold cross-validation. When a specific value for $k$ is chosen, it may be used in place of $k$ in the reference to the model, such as $k$=10 becoming 10-fold cross-validation.

Cross-validation is primarily used in applied machine learning to estimate the skill of a machine learning model on unseen data. That is, to use a limited sample in order to estimate how the model is expected to perform in general when used to make predictions on data not used during the training of the model.

It is a popular method because it is simple to understand and because it generally results in a less biased or less optimistic estimate of the model skill than other methods, such as a simple train/test split.

The general procedure is as follows:

- Shuffle the dataset randomly.
- Split the dataset into $k$ groups
- For each unique group:
    - Take the group as a hold out or test data set
    - Take the remaining groups as a training data set
    - Fit a model on the training set and evaluate it on the test set
    - Retain the evaluation score and discard the model
- Summarize the skill of the model using the sample of model evaluation scores
Importantly, each observation in the data sample is assigned to an individual group and stays in that group for the duration of the procedure. This means that each sample is given the opportunity to be used in the hold out set 1 time and used to train the model $k$-1 times.

Let's keep the default hyperparameters set by the function and only set the values for 'n_estimators' and 'random-state' hyperparameters. This will be our base model for comparison.

In [ ]:
base_model = RandomForestRegressor(n_estimators = 50, random_state = 42)
base_model.fit(df_train, y_train)
y_pred_base,rmse_base,r2_score_base, adjusted_r2_score_base = evaluate(base_model, df_test, y_test)

In [ ]:
base_model

Now, let's set a grid search for 'n_estimators' over a range of values to find the optimal value using a 5-fold cross-validation.

In [ ]:
param_grid = { 
    'n_estimators': list(range(40,64,2))
}

# Use the grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 5 fold cross validation, 
# search across all different combinations, and use all available cores
rf_grid = GridSearchCV(estimator = rf, param_grid = param_grid, scoring='neg_mean_squared_error', cv = 3, verbose=2, n_jobs = -1)
# Fit the random search model
rf_grid.fit(df_train, y_train)

The optimal value found for 'n_estimators' is as follows:

In [ ]:
rf_grid.best_params_

Let's train a random forest regression model based on the best parameter found and evaluate the model fit.

In [ ]:
best_rf = rf_grid.best_estimator_
y_pred_best,rmse_best,r2_score_best,adjusted_r2_score_best = evaluate(best_rf, df_test, y_test)

In [ ]:
best_rf

As observed, there is not much improvement in model performance as compared to the base model. This usually won't be the case all the time. However, the model fit is much better compared to the linear regression model.

#### What if just the preceding year of data is used to train the regression model? (Optional)
Let's experiment and see if using only recent data improves performance as there are no obvious trends to resale prices of HDB flats. We'll repeat for both linear and non-linear for selected columns.

Create the train and test sets. The train set should be for the year 2017 and test for 2018.

In [ ]:
# To do
# Create train set


In [ ]:
# Solution
df_train = df1[(df1['month'].dt.year == 2017)]
y_train = df_train['resale_price']
df_train = df_train.drop(['resale_price','month'], axis = 1)
df_train.head()

In [ ]:
## To do
## Create test set


In [ ]:
# Solution
df_test = df1[(df1['month'].dt.year == 2018)]
y_test = df_test['resale_price']
df_test = df_test.drop(['resale_price','month'], axis = 1)
df_test.head()

Results from linear regression:

In [ ]:
lm = LinearRegression(normalize=False,n_jobs=-1)
lm.fit(df_train,y_train)
# The mean squared error
print("Root mean squared error: %.2f" % np.sqrt(mean_squared_error(y_train, lm.predict(df_train))))
# Explained variance score: 1 is perfect prediction
print("R^{2} score: ", lm.score(df_train,y_train))
# Adjusted R-Squared
print("Adjusted R^{2} score: ", 1 - (1-lm.score(df_train,y_train))*(len(y_train)-1)/(len(y_train)-df_train.shape[1]-1))

In [ ]:
# Make predictions using the testing set
y_pred = lm.predict(df_test)

In [ ]:
# The mean squared error
print("Root mean squared error: %.2f" % np.sqrt(mean_squared_error(y_test, y_pred)))
# Explained variance score: 1 is perfect prediction
print('R^{2} score:', lm.score(df_test, y_test))
# Adjusted R-Squared
print("Adjusted R^{2} score: ", 1 - (1-lm.score(df_test, y_test))*(len(y_test)-1)/(len(y_test)-df_test.shape[1]-1))

In [ ]:
df_predict_gt = pd.DataFrame({'Predicted Price' : y_pred, 'Actual Price' : y_test})
df_predict_gt.head()

Results from random forest regression:

In [ ]:
param_grid = { 
    'n_estimators': list(range(40,64,2))
}

# Use the grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 5 fold cross validation, 
# search across all different combinations, and use all available cores
rf_grid = GridSearchCV(estimator = rf, param_grid = param_grid, scoring='neg_mean_squared_error', cv = 3, verbose=2, n_jobs = -1)
# Fit the random search model
rf_grid.fit(df_train, y_train)

The optimal value found for 'n_estimators' is as follows:

In [ ]:
rf_grid.best_params_

Let's train a random forest regression model based on the best parameter found and evaluate the model fit.

In [ ]:
best_rf = rf_grid.best_estimator_
y_pred_best,rmse_best,r2_score_best,adjusted_r2_score_best = evaluate(best_rf, df_test, y_test)

As observed, for linear regression models, the R-squared values are generally better but the RMSE is higher than that of the model trained using the preceding 5-year data. Similar, observation could be made for the random forest models. However, as mentioned in the discussion above about the assessment of model fit, the most important criterion that trumps is RMSE. Hence, as it is with machine learning use cases, the more data you have for training, the better will be the predicted results.